In [ ]:
import tensorflow as tf
from gensim.models import Word2Vec
import pandas as pd
from textblob import TextBlob
import numpy as np
from tqdm import tqdm

### Importing the word2vec models

In [ ]:
word_vec = Word2Vec.load('model.bin')    # word embedding
# char_vec = Word2Vec.load('model_c.bin')  # charector embedding

### Importing the dataset

In [ ]:
df_contxt = pd.read_pickle('./input/squad_contxt.pkl') # context
df_qas = pd.read_pickle('./input/squad_qas.pkl')       # question and answers
df_qas = df_qas[df_qas.is_impossible != True]

In [ ]:
print(len(df_contxt),len(df_qas))
max_c_word_length = 0
min_c_word_length = 1000
max_c_snt_length = 0
min_c_snt_length = 1000
p_word_length = []
p_snt_length = []
ran = 1
for i in tqdm(df_contxt['context']):
    w_len = 0
    l = len(TextBlob(i).sentences)
    if l < min_c_snt_length:
        min_c_snt_length = l
    if l > max_c_snt_length:
        max_c_snt_length = l   
    p_snt_length.append(l)
    for j in TextBlob(i).sentences:
        k = len((j+" E-O-S").words)
        w_len += k
    p_word_length.append(w_len)
    if w_len > max_c_word_length:
        max_c_word_length = w_len
    if w_len < min_c_word_length:
        min_c_word_length = w_len
#     print(w_len,df_contxt['word_len'][ran])
    ran += 1
print(min_c_word_length,max_c_word_length)
print(min_c_snt_length,max_c_snt_length)
        

In [ ]:
# print(len(df_contxt),len(p_word_length))
df_contxt['snt_len'] = p_snt_length
df_contxt['word_len'] = p_word_length

df_contxt.head()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.hist(p_word_length,bins = 50)
plt.show()

In [ ]:
plt.hist(p_snt_length,bins = 30)
plt.show()

In [ ]:
max_q_word_length = 0
min_q_word_length = 1000
max_q_snt_length = 0
min_q_snt_length = 1000
q_word_len = []
q_snt_len = []
# ran = 0
for i in tqdm(df_qas["Question"]):
    q_word_len.append(len(TextBlob(i).words))
    q_snt_len.append(len(TextBlob(i).sentences))
    if len(TextBlob(i).sentences)>max_q_snt_length:
        max_q_snt_length = len(TextBlob(i).sentences)
    if len(TextBlob(i).sentences)<min_q_snt_length:
        min_q_snt_length = len(TextBlob(i).sentences)
    if len(TextBlob(i).words)<min_q_word_length:
        min_q_word_length = len(TextBlob(i).words)
    if len(TextBlob(i).words)>max_q_word_length:
        max_q_word_length = len(TextBlob(i).words)
df_qas['word_len'] = q_word_len
df_qas['snt_len'] = q_snt_len
#     if len(TextBlob(i).sentenses) >1

In [ ]:
print(min_q_snt_length,max_q_snt_length)
print(min_q_word_length,max_q_word_length)
df_qas.head()

In [ ]:
plt.hist(q_snt_len,bins = 10)
plt.show()

In [ ]:
plt.hist(q_word_len,bins = 30)
plt.show()

In [ ]:
df_contxt = df_contxt[df_contxt.word_len <= 200 ] # removing paras with more than 200 words
df_qas = df_qas[df_qas.snt_len ==1] # removing questions with more than one sentence

In [ ]:
len(df_contxt),len(df_qas)
# df_contxt['context_no']

In [ ]:
for i in df_qas['context_no']:
    if i not in df_contxt['context_no']:
        #print('gotcha !',i)
        df_qas = df_qas[df_qas.context_no != i]
        

### Preprocessing data

In [ ]:
class prep_data():
    """
    
    """
    def __init__(self, word_e = word_vec):
        
        self.contxt = pd.read_pickle('./input/squad_contxt.pkl')
        self.qas = pd.read_pickle('./input/squad_qas.pkl')
#         self.char_e = char_e
        self.word_e = word_e

    def C2V(self,q_no):   # this function converts context 2 vector with <EOS> tag
        tb_p = TextBlob(self.context['Context'][self.qas['context_no'][q_no]])#TextBlob(passage)
        tb_q = TextBlob(self.qas['Question'][q_no])
        p2v = []
        q2v = []
        eos_tag = []
        c = 0
        for i in tb_p.sentences:
            words = TextBlob(str(i)+" E-O-S").words
            for j in words:
                p2v.append(self.word_e[j])
                if j=='E-O-S':
                    eos_tag.append(c)
            c += 1
        for k in (str(tb_q)+" E-O-S").words:
            q2v.append(self.word_e[k])
            
        return np.array(p2v),np.array(q2v),eos_tag
    
#     def QA2V(self,q_no):  # this function converts question 2 vector
#         tb_a = TextBlob(self)
        
        
        

###  Building the model

In [ ]:
class gru_models():
    """
        It's a modified version of normal GRU
    """
    def __init__(self,time_steps,init_state,input_size,tag,
                 output_size,question=None,final_mem=None,input_seq=None):
        
        # inputs required for sequential memory vectors e_i 
        self.time_steps = time_steps              # equal to max no concepts
        self.init_state = init_state              # initial secondary input ideally zero tensor
        self.input_seq = input_seq                # input seq to be evaluated
        self.input_size = input_size              # size of input vector 
        self.output_size = output_size            # size of output required
        
        # inputs required for Answer modules
        self.question = question
        self.final_mem = final_mem
        self.word_weight = tf.get_variable("word_w_{}".format(tag),[output_size,output_size],dtype=tf.float64)
        self.word_bias = tf.get_variable("word_b_{}".format(tag),[output_size,1],dtype=tf.float64)
        
        # parameters of gru
        self.W_z,self.W_r,self.W_h = tf.get_variable("update_w_{}".format(tag),[output_size,input_size],dtype=tf.float64),tf.get_variable("reset_w_{}".format(tag),[output_size,input_size],dtype=tf.float64),tf.get_variable("out_w_{}".format(tag),[output_size,input_size],dtype=tf.float64)
        self.U_z,self.U_r,self.U_h = tf.get_variable("update_u_{}".format(tag),[output_size,output_size],dtype=tf.float64),tf.get_variable("reset_u_{}".format(tag),[output_size,output_size],dtype=tf.float64),tf.get_variable("out_u_{}".format(tag),[output_size,output_size],dtype=tf.float64)
        self.b_z,self.b_r,self.b_h = tf.get_variable("update_b_{}".format(tag),[output_size,1],dtype=tf.float64),tf.get_variable("reset_b_{}".format(tag),[output_size,1],dtype=tf.float64),tf.get_variable("out_b_{}".format(tag),[output_size,1],dtype=tf.float64)
    
    """ Normal GRU unit """
    def gru_unit(self,h_prev,steps,out=[]):
        index = steps - self.time_steps 
        input_vector = self.input_seq[index]
        # operations 
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        out.append(h_t)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.gru_unit(h_prev = h_t,out = out,steps = steps) 
    
    
    """
    it's a modified GRU !!!
    """
    def mod_gru_unit(self,h_prev,steps,scalar_values,out=[],index=0):
        input_vector = self.input_seq[index]
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        h_t = (scalar_values[index]*h_t) + ((1-scalar_values[index])*h_prev)
        out.append(h_t)
        index += 1
        if steps-index == 0:
            return(out)
        else:
            return self.mod_gru_unit(h_prev = h_t,out = out,steps = steps,
                                     scalar_values=scalar_values,index=index) 
        
    """ It's a hybrid GRU for answer module """
    def answer_gru_unit(self,h_prev,word_prev,out=[]):
        input_vector = tf.concat([word_prev,self.question],axis=0)
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.multiply(z_t,h_prev)+tf.multiply(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(self.U_h,tf.multiply(r_t,h_prev))+self.b_h))
        word_pred = tf.nn.softmax(tf.matmul(self.word_weight,h_t)+self.word_bias)
        out.append(word_pred)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return self.answer_gru_unit(h_prev = h_t,word_prev = word_pred,out = out) 

In [ ]:
class DMN_QA():
    """
    the word vectors have size 100
    the max no of words in a sentence is 120
    """
    
    def __init__(self):
        self.MAX_P_WORDS = 120       # max no of words in a passage
        self.WORD_VEC_LEN = 100      # word vector length
        self.MAX_Q_WORDS = 10        # max no of words in question
        self.MAX_NO_CONCEPTS = 5      # max no of concepts
        
        # variables and placeholders
        self.Passage = tf.placeholder(shape=(self.MAX_P_WORDS,self.WORD_VEC_LEN,1),dtype = tf.float64)
#         self.EOS_Tag = tf.placeholder(tf.float64)
        self.Question = tf.placeholder(shape=(self.MAX_Q_WORDS,self.WORD_VEC_LEN,1),dtype = tf.float64) #sequence for question
#         self.Answer = tf.placeholder(tf.float64)

        # weigts for scalar kernel and memory vector
        self.W_b = tf.get_variable("W_b",shape=(100,100),dtype=tf.float64)
        self.W_1 = tf.get_variable("W_1",shape=(200,702),dtype=tf.float64)
        self.b_1 = tf.get_variable("b_1",shape=(200,1),dtype=tf.float64)
        self.W_2 = tf.get_variable("W_2",shape=(1,200),dtype=tf.float64)
        self.b_2 = tf.get_variable("b_2",shape=(1,1),dtype=tf.float64)
        
        # initial memory state
#         self.memory = tf.placeholder(shape=(100,1),dtype=tf.float64)
        
        #training parameters
        self.no_epoch = 10
        self.df_contxt = pd.read_pickle('./input/squad_contxt.pkl') # context
        self.df_qas = pd.read_pickle('./input/squad_qas.pkl')       # question and answers
        
    def scalar_gate_value(self,concept):  # debugged and perfect
        z_vector = tf.concat((concept,self.memory),axis=0)
        z_vector = tf.concat((z_vector,self.question),axis=0)
        z_vector = tf.concat((z_vector,tf.multiply(concept,self.question)),axis=0)
        z_vector = tf.concat((z_vector,tf.multiply(concept,self.memory)),axis=0)
        z_vector = tf.concat((z_vector,concept-self.question),axis=0)
        z_vector = tf.concat((z_vector,concept-self.memory),axis=0)
        z_vector = tf.concat((z_vector,tf.matmul(tf.transpose(concept),tf.matmul(self.W_b,self.question))),axis=0)
        z_vector = tf.concat((z_vector,tf.matmul(tf.transpose(concept),tf.matmul(self.W_b,self.memory))),axis=0)
        g_1 =  tf.nn.tanh(tf.add(tf.matmul(self.W_1,z_vector),self.b_1))
        g_scalar =  tf.nn.sigmoid(tf.add(tf.matmul(self.W_2,g_1),self.b_2))
        return g_scalar
        
    def epi_mem_mod(self,concepts,no_of_iterations,mode,mem_states=[]): #debugged
        self.scalar_vector = []
        for i in concepts:
            self.scalar_vector.append(self.scalar_gate_value(i))
        
        concepts_out = mode.mod_gru_unit(h_prev = self.memory,steps=len(concepts),scalar_values=self.scalar_vector,
                          out=[])
        self.memory = concepts_out[-1]
        mem_states.append(concepts_out[-1])

        if no_of_iterations != 0:
            self.epi_mem_mod(concepts,no_of_iterations-1,
                        mode,mem_states)

        return mem_states
    
    def get_sentence_weight(self,q_no):
        y = self.df_contxt['context'][self.df_qas['context_no'][q_no]]
        c = -1
        count = 0
        snt_wt = np.zeros(self.MAX_NO_CONCEPTS)
        if self.df_qas['is_impossible'][q_no]!= True:
            print()
            for i in TextBlob(y).sentences:
                c += (len(i)+1)
                if self.df_qas['Answer_start'][q_no] < c:
                    snt_wt[count] = 1.
                    break
        return snt_wt
    
    def build(self):
        
        # question module 
        Q_module = gru_models(time_steps=self.MAX_Q_WORDS,
                              init_state=np.zeros([self.WORD_VEC_LEN,1]),
                              input_size=100,output_size=100,
                              input_seq=self.Question,tag=1)
        Q_out = Q_module.gru_unit(h_prev=np.zeros([self.WORD_VEC_LEN,1]),
                                  steps=self.MAX_Q_WORDS,out=[])
        
        self.question = Q_out[-1] # final representation of question
        #passage module
        P_module = gru_models(time_steps=self.MAX_P_WORDS,
                              init_state=np.zeros([self.WORD_VEC_LEN,1]),
                              input_size=100,output_size=100,
                              input_seq=self.Passage,tag=2)
        P_out = P_module.gru_unit(h_prev=np.zeros([self.WORD_VEC_LEN,1]),steps=self.MAX_P_WORDS,out=[])
        #episodic memory module
        concepts = P_out[:3]  # selected E-O-S tags from P_out
        epi_mod = gru_models(time_steps=2,
                             init_state=np.zeros([self.WORD_VEC_LEN,1]),
                             input_size=100,output_size=100,
                             input_seq=concepts,tag=3)
        self.mem_states = self.epi_mem_mod(concepts,10,epi_mod,[])
        
        self.output = self.scalar_vector
        
        return self.output
    
    def train_model(self):
        pred = self.build()
        cost = self.snt_wt - pred
        init_op = tf.global_variables_initializer()
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(cost)
        with tf.Session() as sess:
            sess.run(init_op)
            for i in range(self.no_epoch):
                c = 0
                while c<1000:
                    q_no  = np.random.randint(0,10000)
                    sess.run(optimizer,feed_dict={self.Passage: self.df_contxt['context'][self.df_qas['context_no'][q_no]] ,
                                                 self.Question: self.df_qas['Question'][q_no] ,
                                                 self.})
                    
                    
            
            
        
    
        """
        #answer module
        A_module = gru_models(time_steps=len(self.mem_states),
                              init_state=None,
                              question=self.question,
                              input_size=200,output_size=100,
                              input_seq=self.mem_states,tag=4)
        A_out = A_module.answer_gru_unit(h_prev=self.mem_states[-1],
                                         word_prev=np.zeros([self.WORD_VEC_LEN,1]),
                                         out=[]) 
        """
                
#         print(A_out)       
        
        

In [ ]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    x = DMN_QA()
    y = x.build()
    
print(y)

In [ ]:
def QA2V(context_no,question_no):
    passage = TextBlob(df_contxt['context'][context_no])
    question = TextBlob(df_qas['Question'][question_no])
    if df_qas['is_impossible'][question_no]==False:
        answer = TextBlob(df_qas['Answer_text'][question_no])
        beginning = passage[:df_qas['Answer_start'][question_no]]
        end = passage[df_qas['Answer_start'][question_no]+len(answer):]  # verified
        print(answer,'\n',beginning,'\n',end)
    print(len(passage.words),'\n',len(beginning.words),'\n',len(answer.words),'\n',len(end.words))
        
    

In [ ]:
# QA2V(1000,1)

In [ ]:
df_contxt['context'][3]

In [ ]:
q_no = 200
x = df_qas[['Question','context_no','q_no','Answer_start','Answer_text','is_impossible']]
print(df_contxt['context'][x['context_no'][q_no]],'\n\n',
      x['Question'][q_no],'\n\n',
      x['Answer_text'][q_no],'\n\n',
      x['Answer_start'][q_no])

In [ ]:
y = TextBlob(df_contxt['context'][x['context_no'][q_no]]).sentences
k = len(df_contxt['context'][x['context_no'][q_no]])
c = -1
for i in y:
#     print('\n',i,len(i))
    c += len(i)+1
print(c,k)

In [ ]:
df_qas.keys()

In [ ]:
df_contxt.keys()

In [ ]:
p = TextBlob('I am Prince.')
p[0:]